3.-Divide aleatoriamente el conjunto de datos de cáncer de seno de Wisconsin3

en un subconjunto de
entrenamiento con el 60 % de los datos, un subconjunto de validación con el 20 % y un subconjunto
de prueba con el 20 % restante usando 0 como semilla para tu generador de números aleatorios.
Este conjunto de datos contiene 699 registros de tumores de seno, de los cuales 458 son benignos y
241 son malignos.

In [28]:
#Librerias
import pandas as pd
import matplotlib.pyplot as plt  

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

In [2]:

# URL del archivo de datos
url = "http://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/breast-cancer-wisconsin.data"

# Cargar los datos en un DataFrame
df = pd.read_csv(url, header=None)  # No tiene encabezados, por eso usamos header=None


In [3]:
# Definir nombres de columnas en español
column_names = [
    "ID_Muestra", "Grosor_Tumor", "Uniformidad_Tamaño_Celular", 
    "Uniformidad_Forma_Celular", "Adhesión_Marginal", "Tamaño_Célula_Epitelial", 
    "Núcleos_Desnudos", "Cromatina_Blanda", "Nucléolos_Normales", 
    "Mitosis_Celular", "Clase"
]

# Asignar los nombres de columna
df.columns = column_names

In [4]:
# corroboramos el tipo de datos
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 699 entries, 0 to 698
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   ID_Muestra                  699 non-null    int64 
 1   Grosor_Tumor                699 non-null    int64 
 2   Uniformidad_Tamaño_Celular  699 non-null    int64 
 3   Uniformidad_Forma_Celular   699 non-null    int64 
 4   Adhesión_Marginal           699 non-null    int64 
 5   Tamaño_Célula_Epitelial     699 non-null    int64 
 6   Núcleos_Desnudos            699 non-null    object
 7   Cromatina_Blanda            699 non-null    int64 
 8   Nucléolos_Normales          699 non-null    int64 
 9   Mitosis_Celular             699 non-null    int64 
 10  Clase                       699 non-null    int64 
dtypes: int64(10), object(1)
memory usage: 60.2+ KB


In [5]:
#La columna tiene un tipo de dato objecto, veremos que valores extra a numeros tiene.
columna = "Núcleos_Desnudos"
valores_no_numericos = df[pd.to_numeric(df[columna], errors="coerce").isna()]
print(valores_no_numericos[columna])

23     ?
40     ?
139    ?
145    ?
158    ?
164    ?
235    ?
249    ?
275    ?
292    ?
294    ?
297    ?
315    ?
321    ?
411    ?
617    ?
Name: Núcleos_Desnudos, dtype: object


In [6]:
#Encontramos 16 valores "?" en la columna "Núcleos_Desnudos" por lo que prcedemos a remplazarlos por 
#NANs y poder trabajar regularmente con ellos 
df.replace("?", pd.NA, inplace=True)

#Convertimos la columna a tipo entero
df["Núcleos_Desnudos"] = pd.to_numeric(df["Núcleos_Desnudos"]).astype("Int64")




In [7]:
#Validamos los 16 nans
print(df.isna().sum().sum())

16


In [8]:
#Validamos si el volumen de los datos coincide con la información proporcionada
pivot_table = df.pivot_table(values="ID_Muestra", index="Clase", aggfunc="count")
pivot_table.loc["Total"] = pivot_table.sum()
pivot_table["Porcentaje"] = (pivot_table["ID_Muestra"] / pivot_table.loc["Total", "ID_Muestra"]) * 100
pivot_table

,ID_Muestra,Porcentaje
Clase,,
2,458,65.522175
4,241,34.477825
Total,699,100.000000


In [9]:
#quitaremosw el ID de la muestra ya que no aporta información relevante
df.drop(columns="ID_Muestra", inplace=True)

Usaremos 3 metodologias para el tratamiento de los datos faltantes

1. Eliminar las filas con datos faltantes
2. Imputar los datos faltantes con la media
3. Interpolación lineal para los datos faltantes

In [10]:
#1 eliminar las filas con valores nulos 
df_drop = df.dropna()

#2 reemplazar los valores nulos con la media de la columna
df_avg = df
df_avg["Núcleos_Desnudos"] = df_avg["Núcleos_Desnudos"].fillna(round(df_avg["Núcleos_Desnudos"].mean(),0)).astype("int64")

#3 haremos una interpolación lineal
df_itrp = df.interpolate(method="linear")

In [18]:
#función para extraer los grupos requeridos

def separar(df):


    X = df.drop(columns=['Clase'])  
    y = df['Clase']  

    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=0)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=0)
    return X_train, X_val, X_test, y_train, y_val, y_test

In [19]:
#Podemos generar cada crupo de datos
X_train, X_val, X_test, y_train, y_val, y_test = separar(df_drop)
X_train2, X_val2, X_test2, y_train2, y_val2, y_test2 = separar(df_avg)
X_train3, X_val3, X_test3, y_train3, y_val3, y_test3 = separar(df_itrp)


In [12]:
#Regresión Logística
#Support Vector Machines (SVM)
#Random Forest
#Naive Bayes

In [29]:
#LOGISTICA

# Estandarizar las características
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Crear y entrenar el modelo de regresión logística
model = LogisticRegression()
model.fit(X_train_scaled, y_train)

# Predicción
y_pred = model.predict(X_test_scaled)

# Evaluación del modelo
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy:.4f}')
print('Confusion Matrix:')
print(conf_matrix)
print('Classification Report:')
print(class_report)

Accuracy: 0.9635
Confusion Matrix:
[[80  4]
 [ 1 52]]
Classification Report:
              precision    recall  f1-score   support

           2       0.99      0.95      0.97        84
           4       0.93      0.98      0.95        53

    accuracy                           0.96       137
   macro avg       0.96      0.97      0.96       137
weighted avg       0.96      0.96      0.96       137

